## Module_2

## Team Members:
Daniel and Aarnav

## Project Title:
Lung Depth and Fibrosis Intensity



## Project Goal:
This project seeks to analyze tissue damage by idiopathic pulmonary fibrosis at different depths of the lung. 

## Disease Background: 
*Fill in information and please note that this module is truncated and only has 5 bullets (instead of the 11 that you did for Module #1).*

* Prevalence & incidence: 
    
    * In the US, there are 50,000 new diagnosed cases of idiopathic pulmonary fibrosis
* Risk factors (genetic, lifestyle): https://www.nhlbi.nih.gov/health/idiopathic-pulmonary-fibrosis/causes
    *  Lung Fibrosis has an increased risk of devlopment as age increases. 
    *  Since the disease causes scar tissue on the lower lungs, it is based off of weakening of the repair process after excessive damage. Further risk factors that may cause excessive damage are smoking, specific genetics (either by mutation or family iniheritance), respiratory infections. 
    *  Based on previous studies, it is seen that men diagnosis is more common than female. 
* Symptoms: https://www.ncbi.nlm.nih.gov/books/NBK448162/ and https://www.mayoclinic.org/diseases-conditions/pulmonary-fibrosis/symptoms-causes/syc-20353690 
    * Linked with common cough, fatigue, aching muscles (back) and shortness of breath
    * Increased risk of other respiratory or musculoskeletal diseases. Exampeles being venous thromboembolism, gastroesophageal reflux disease, or osteoporosis.
* Standard of care treatment(s)
* Biological mechanisms (anatomy, organ physiology, cell & molecular physiology)

## Data-Set: 
*(Describe the data set(s) you will analyze. Cite the source(s) of the data. Describe how the data was collected -- What techniques were used? What units are the data measured in? Etc.)*

The data-set analyzed was provided by Dr. Peirce-Cottler in research imaging lung tissue. Specifically, the images portay a 2D cut out of the lungs at different depths (ranging from 15 to 10000 micrometers) to visualize tissue damage. Shown within these jpg.'s, the white pixelation represents either tissue damage or misreads of the circulation system within the lungs. By splitting each of these up and organizing them by their respective depths, the data analysis sets to match a percentage of all the white pixels to each image. 

## Data Analyis: 
*(Describe how you analyzed the data. This is where you should intersperse your Python code so that anyone reading this can run your code to perform the analysis that you did, generate your figures, etc.)*

'''Module 2: count black and white pixels in a .jpg and extrapolate points'''

from termcolor import colored
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import pandas as pd

# Load the images you want to analyze

filenames = [
    r"C:\Users\aarna\Documents\UVA\BME 2315 - Computational BME\module2\MASK_Sk658 Llobe ch010033.jpg",
    r"C:\Users\aarna\Documents\UVA\BME 2315 - Computational BME\module2\MASK_Sk658 Llobe ch010034.jpg",
    r"C:\Users\aarna\Documents\UVA\BME 2315 - Computational BME\module2\MASK_Sk658 Llobe ch010035.jpg",
    r"C:\Users\aarna\Documents\UVA\BME 2315 - Computational BME\module2\MASK_Sk658 Llobe ch010036.jpg",
    r"C:\Users\aarna\Documents\UVA\BME 2315 - Computational BME\module2\MASK_Sk658 Llobe ch010039.jpg",
    r"C:\Users\aarna\Documents\UVA\BME 2315 - Computational BME\module2\MASK_Sk658 Llobe ch010040.jpg",
            ]

# Enter the depth of each image (in the same order that the images are listed above; you can find these in the .csv file provided to you which is tilted: "Filenames and Depths for Students")

depths = [
    2950,
    6600,
    780,
    9200,
    15,
    380
    ]

# Make the lists that will be used

images = []
white_counts = []
black_counts = []
white_percents = []

# Build the list of all the images you are analyzing

for filename in filenames:
    img = cv2.imread(filename, 0)
    images.append(img)

# For each image (until the end of the list of images), calculate the number of black and white pixels and make a list that contains this information for each filename.

for x in range(len(filenames)):
    _, binary = cv2.threshold(images[x], 127, 255, cv2.THRESH_BINARY)

    white = np.sum(binary == 255)
    black = np.sum(binary == 0)

    white_counts.append(white)
    black_counts.append(black)

# Print the number of white and black pixels in each image.

print(colored("Counts of pixel by color in each image", "yellow"))
for x in range(len(filenames)):
    print(colored(f"White pixels in image {x}: {white_counts[x]}", "white"))
    print(colored(f"Black pixels in image {x}: {black_counts[x]}", "black"))
    print()

# Calculate the percentage of pixels in each image that are white and make a list that contains these percentages for each filenamae

for x in range(len(filenames)):
    white_percent = (100 * (white_counts[x] / (black_counts[x] + white_counts[x])))
    white_percents.append(white_percent)

# Print the filename (on one line in red font), and below that line print the percent white pixels and depth

print(colored("Percent white px:", "yellow"))
for x in range(len(filenames)):
    print(colored(f'{filenames[x]}:',"red"))
    print(f'{white_percents[x]}% White | Depth: {depths[x]} microns')
    print()

'''Write your data to a .csv file'''

# Create a DataFrame that includes the filenames, depths, and percentage of white pixels
df = pd.DataFrame({
    'Filenames': filenames,
    'Depths': depths,
    'White percents': white_percents       
})

# Write that DataFrame to a .csv file

df.to_csv('Percent_White_Pixels.csv', index=False)

print("CSV file 'Percent_White_Pixels.csv' has been created.")

'''the .csv writing subroutine ends here'''

# Interpolate a point: given a depth, find the corresponding white pixel percentage

interpolate_depth = float(input(colored("Enter the depth at which you want to interpolate a point: ", "yellow")))

x = depths
y = white_percents

i = interp1d(x, y, kind='linear')  # You can also use 'quadratic', 'cubic', etc.
interpolate_point = i(interpolate_depth)
print(colored(f'The interpolated point is at the x-coordinate {interpolate_depth} and y-coordinate {interpolate_point}.', "green"))

depths_i = depths[:]
depths_i.append(interpolate_depth)
white_percents_i = white_percents[:]
white_percents_i.append(interpolate_point)

# make two plots: one that doesn't contain the interpolated point, just the data calculated from your images, and one that also contains the interpolated point (shown in red)
fig, axs = plt.subplots(2, 1)

axs[0].scatter(depths, white_percents, marker='o', linestyle='-', color='blue')
axs[0].set_title('Plot of depth of image vs percentage white pixels')
axs[0].set_xlabel('depth of image')
axs[0].set_ylabel('white pixels as a percentage of total pixels')
axs[0].grid(True)


axs[1].scatter(depths_i, white_percents_i, marker='o', linestyle='-', color='blue')
axs[1].set_title('Plot of depth of image vs percentage white pixels w/ interpolated point (red)')
axs[1].set_xlabel('depth of image')
axs[1].set_ylabel('white pixels as a percentage of total pixels')
axs[1].grid(True)
axs[1].scatter(depths_i[len(depths_i)-1], white_percents_i[len(white_percents_i)-1], color='red', s=100, label='Highlighted point')


# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

## Verify and validate your analysis: 
*(Describe how you checked to see that your analysis gave you an answer that you believe (verify). Describe how your determined if your analysis gave you an answer that is supported by other evidence (e.g., a published paper).*

## Conclusions and Ethical Implications: 
*(Think about the answer your analysis generated, draw conclusions related to your overarching question, and discuss the ethical implications of your conclusions.*

## Limitations and Future Work: 
*(Think about the answer your analysis generated, draw conclusions related to your overarching question, and discuss the ethical implications of your conclusions.*

## NOTES FROM YOUR TEAM: 
*This is where our team is taking notes and recording activity.*

## QUESTIONS FOR YOUR TA: 
*These are questions we have for our TA.*